In [1]:
!pip install stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
import sys
import os

# Add the directory where custom_env.py is located
sys.path.append('/content/')


In [3]:
import os
import numpy as np
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
from stable_baselines3.common.logger import configure
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from custom_env import EducationHubEnv  # Your fixed custom env

# Custom callback to log rewards
class EducationLoggerCallback(BaseCallback):
    def __init__(self, log_dir, verbose=1):
        super(EducationLoggerCallback, self).__init__(verbose)
        self.log_dir = log_dir
        os.makedirs(log_dir, exist_ok=True)
        self.episode_rewards = []

    def _on_step(self) -> bool:
        if "infos" in self.locals and len(self.locals["infos"]) > 0:
            episode_reward = self.locals["infos"][0].get("episode", {}).get("r")
            if episode_reward is not None:
                self.episode_rewards.append(episode_reward)
                print(f"Episode Reward: {episode_reward}")
        return True

    def on_training_end(self):
        np.save(os.path.join(self.log_dir, "dqn_rewards.npy"), np.array(self.episode_rewards))

def train_dqn():
    # ✅ Wrap environment properly for SB3
    env = DummyVecEnv([lambda: Monitor(EducationHubEnv())])

    # Define paths
    models_dir = "models/dqn/"
    log_dir = "logs/dqn/"
    best_model_dir = os.path.join(models_dir, "best/")
    os.makedirs(models_dir, exist_ok=True)
    os.makedirs(log_dir, exist_ok=True)
    os.makedirs(best_model_dir, exist_ok=True)

    # ✅ DQN Model with tuned hyperparameters
    model = DQN(
        "MlpPolicy",
        env,
        verbose=1,
        learning_rate=1e-4,
        gamma=0.99,
        batch_size=64,
        buffer_size=100000,
        exploration_initial_eps=1.0,
        exploration_final_eps=0.02,
        exploration_fraction=0.2,
        tensorboard_log=log_dir
    )

    # ✅ Configure logger (stdout + tensorboard)
    logger = configure(log_dir, ["stdout", "tensorboard"])
    model.set_logger(logger)

    # ✅ Callbacks
    reward_logger = EducationLoggerCallback(log_dir)
    eval_callback = EvalCallback(
        env,
        best_model_save_path=best_model_dir,
        eval_freq=5000,
        deterministic=True,
        render=False
    )

    # ✅ Train!
    model.learn(
        total_timesteps=100000,
        callback=[reward_logger, eval_callback]
    )

    # ✅ Save final model
    model.save(os.path.join(models_dir, "dqn_final_model"))
    print("Training complete. Models saved to:", models_dir)

    # ✅ Close environment
    env.close()

if __name__ == "__main__":
    train_dqn()

/usr/local/lib/python3.11/dist-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/usr/local/lib/python3.11/dist-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


Streaming output truncated to the last 5000 lines.
| rollout/            |          |
|    ep_len_mean      | 6        |
|    ep_rew_mean      | 1.19     |
|    exploration_rate | 0.02     |
| time/               |          |
|    episodes         | 15616    |
|    fps              | 597      |
|    time_elapsed     | 156      |
|    total_timesteps  | 93732    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 2.25e-06 |
|    n_updates        | 23407    |
----------------------------------
Episode Reward: 1.4
Episode Reward: 1.4
Episode Reward: 1.4
Episode Reward: 1.4
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 6        |
|    ep_rew_mean      | 1.19     |
|    exploration_rate | 0.02     |
| time/               |          |
|    episodes         | 15620    |
|    fps              | 597      |
|    time_elapsed     | 156      |
|    total_timesteps  | 93756    |
| train/              |      